In [9]:
import csv
import json
import gzip
import shutil
import os
import random
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import JSONFormatter
import tiktoken

In [10]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [17]:
def temp():
    # file_number = random.randint(0, 1023)
    # file_number_str = f"{file_number:04d}"
    file_number_str = "0000"


    # Build the input file name
    input_file_name = f"./../../metadata/yttemporal1b_train_{file_number_str}of1024.jsonl.gz"
    # Build the temporary unzipped file name
    unzipped_file_name = f"./../../metadata/yttemporal1b_train_{file_number_str}of1024.jsonl"

    # Extract the JSONL file from the gzip file
    with gzip.open(input_file_name, 'rb') as gz_file:
        with open(unzipped_file_name, 'wb') as json_file:
            shutil.copyfileobj(gz_file, json_file)


    # Read all the video data from the JSON Lines file into a list
    with open(unzipped_file_name, "r") as input_file:
        datas = [json.loads(line) for line in input_file]

        output_file_path = f'''./../../sampleData/{file_number_str}-5-7/metadata.json'''
        if not os.path.exists(f'''./../../sampleData/{file_number_str}-5-7'''):
            os.mkdir(f'''./../../sampleData/{file_number_str}-5-7''')

        for i in range(0, len(datas) - 1):
            cur_vid = datas[i]
            if(cur_vid["duration"] > 420):
                try:
                    srt = YouTubeTranscriptApi.get_transcript(cur_vid["id"], languages=['en'])
                    formatter = JSONFormatter()
                    json_formatted = formatter.format_transcript(srt)
                except Exception as e:
                    continue
                else:
                    if num_tokens_from_string(json_formatted, "gpt-4") < 7500:
                        with open(output_file_path, "a+") as output_file:
                            json.dump(cur_vid, output_file)
                            output_file.write("\n")

In [ ]:
temp()

In [4]:
la = "./../../sampleData/0000-5-7/metadata.json"

with open(la, "r") as input_file:
    datas = [json.loads(line) for line in input_file]
    print(len(datas))

# Now, data_list contains a list of individual JSON objects



1812
